In [ ]:
pip install tabulate
!pip install rank_bm25
#pip install --force-reinstall -q -U bitsandbytes transformers peft accelerate torch




# **> falcon-7b-instruct**




In [ ]:
import time
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from torch.nn.functional import softmax
from tabulate import tabulate

# -------------------------
# 1. Load Falcon Model
# -------------------------
MODEL_NAME = "tiiuae/falcon-7b-instruct"   # open-source Falcon-7B variant

#with open("test_data.json", "r", encoding="utf-8") as f:
   # test_data = json.load(f)


print("Loading model... (this may take a while)")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.bfloat16
)

# -------------------------
# 2. Define test Q&A pairs
# -------------------------
test_data = [
    {
        "question": "What was the assets current assets cash and cash equivalents in 2023?",
        "answer": "For the year 2023, the assets current assets cash and cash equivalents was $1,481 million."
    },
    {
        "question": "What was the assets current assets cash and cash equivalents in 2022?",
        "answer": "For the year 2022, the assets current assets cash and cash equivalents was $2,305 million."
    },
    {
        "question": "What was the current investments trade receivables in 2023?",
        "answer": "For the year 2023, the current investments trade receivables was $3,094 million."
    },
    {
        "question": "What was the current investments trade receivables in 2022?",
        "answer": "For the year 2022, the current investments trade receivables was $2,995 million."
    },
    {
        "question": "What was the unbilled revenues in 2023?",
        "answer": "For the year 2023, the unbilled revenues was $1,861 million."
    },
    {
        "question": "What was the unbilled revenues in 2022?",
        "answer": "For the year 2022, the unbilled revenues was $1,526 million."
    },
    {
        "question": "What was the prepayments and other current assets in 2023?",
        "answer": "For the year 2023, the prepayments and other current assets was $1,336 million."
    },
    {
        "question": "What was the prepayments and other current assets in 2022?",
        "answer": "For the year 2022, the prepayments and other current assets was $1,133 million."
    },
    {
        "question": "What was the income tax assets derivative financial instruments total current assets in 2023?",
        "answer": "For the year 2023, the income tax assets derivative financial instruments total current assets was $8,626 million."
    },
    {
        "question": "What was the income tax assets derivative financial instruments total current assets in 2022?",
        "answer": "For the year 2022, the income tax assets derivative financial instruments total current assets was $8,865 million."
    },
    {
        "question": "What was the non-current assets property, plant and equipment in 2023?",
        "answer": "For the year 2023, the non-current assets property, plant and equipment was $1,679 million."
    },
    {
        "question": "What was the non-current assets property, plant and equipment in 2022?",
        "answer": "For the year 2022, the non-current assets property, plant and equipment was $1,793 million."
    },
    {
        "question": "What was the right-of-use assets goodwill intangible assets non-current investments in 2023?",
        "answer": "For the year 2023, the right-of-use assets goodwill intangible assets non-current investments was $1,530 million."
    }
]

# -------------------------
# 3. Helper functions
# -------------------------
def get_model_answer(question, max_new_tokens=50):
    """Generate answer + confidence + time."""
    inputs = tokenizer(question, return_tensors="pt").to(model.device)

    start = time.time()
    outputs = model.generate(**inputs, max_new_tokens=max_new_tokens, return_dict_in_generate=True, output_scores=True)
    end = time.time()

    # Decode model output
    answer = tokenizer.decode(outputs.sequences[0], skip_special_tokens=True)
    answer = answer.replace(question, "").strip()  # clean echo

    # Confidence (average max prob across generated tokens)
    scores = torch.stack(outputs.scores, dim=1)  # (batch, seq_len, vocab_size)
    probs = softmax(scores, dim=-1)
    max_probs = probs.max(-1).values.mean().item()

    return answer, max_probs, end - start

def is_correct(pred, truth):
    """Check if ground truth string is in prediction."""
    return truth.lower() in pred.lower()

# -------------------------
# 4. Run evaluation
# -------------------------
results = []
correct_count = 0


for item in test_data[:10]:  # limit to 10 for benchmarking
    question = item["question"]
    truth = item["answer"]
    answer, conf, duration = get_model_answer(question)
    correct = is_correct(answer, truth)
    correct_count += int(correct)

    results.append([question, truth, answer[:60] + ("..." if len(answer) > 60 else ""),
                    f"{conf:.2f}", f"{duration:.2f}", "Y" if correct else "N"])

# -------------------------
# 5. Print results table
# -------------------------
headers = ["Question", "Ground Truth", "Model Answer", "Confidence", "Time (s)", "Correct"]
print("\nBaseline Evaluation (Falcon-7B)\n")
print(tabulate(results, headers=headers, tablefmt="grid"))

accuracy = correct_count / len(test_data)
print(f"\nOverall Accuracy: {accuracy:.2%}")

Loading model... (this may take a while)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/4.48G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.



Baseline Evaluation (Falcon-7B)

+-----------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------+--------------+------------+-----------+
| Question                                                                                      | Ground Truth                                                                                                       | Model Answer                                                    |   Confidence |   Time (s) | Correct   |
+===============================================================================================+====================================================================================================================+=================================================================+==============+============+===========+
| W



# **> MODEL_NAME = "distilbert-base-uncased"**



In [ ]:
import time
import torch
from transformers import pipeline, AutoTokenizer
from tabulate import tabulate
import json

# Define the model name for DistilBERT
MODEL_NAME = "distilbert-base-uncased"

# For extractive QA models, the context needs to be well-structured
# to help the model find the correct answer span. A single, long
# string of disjointed facts is difficult for the model to process.
# We will use a more structured context here.
CONTEXT = """
The company's financial results for the years 2023 and 2022 are as follows:
- Assets:
- Cash and cash equivalents: $1,481 million in 2023 and $2,305 million in 2022.
- Current investments trade receivables: $3,094 million in 2023 and $2,995 million in 2022.
- Unbilled revenues: $1,861 million in 2023 and $1,526 million in 2022.
- Prepayments and other current assets: $1,336 million in 2023 and $1,133 million in 2022.
- Income tax assets, derivative financial instruments, and total current assets: $8,626 million in 2023 and $8,865 million in 2022.
- Non-current assets property, plant and equipment: $1,679 million in 2023 and $1,793 million in 2022.
- Right-of-use assets, goodwill, intangible assets, and non-current investments: $1,530 million in 2023.
"""

# Load the question-answering pipeline
print("Loading DistilBERT model for Question Answering... (this will be fast)")
qa_pipeline = pipeline(
    "question-answering",
    model=MODEL_NAME,
    tokenizer=MODEL_NAME,
    device=0 if torch.cuda.is_available() else -1
)
print("Model loaded.")

# Define your test Q&A pairs
test_data = [
    {"question": "What was the assets current assets cash and cash equivalents in 2023?", "answer": "$1,481 million."},
    {"question": "What was the unbilled revenues in 2022?", "answer": "$1,526 million."},
    {"question": "What was the non-current assets property, plant and equipment in 2023?", "answer": "$1,679 million."},
    {"question": "What was the total current assets in 2022?", "answer": "$8,865 million."}
]

# Helper function to get model answer with confidence
def get_model_answer(question):
    """
    Generate answer from the context using the QA pipeline.
    """
    start = time.time()
    result = qa_pipeline(question=question, context=CONTEXT)
    end = time.time()

    answer = result['answer']
    confidence = result['score']
    duration = end - start

    return answer, confidence, duration

def is_correct(pred, truth):
    """Check if ground truth string is in prediction."""
    return truth.lower() in pred.lower()

# Run evaluation
results = []
correct_count = 0

for item in test_data:
    question = item["question"]
    truth = item["answer"]
    answer, conf, duration = get_model_answer(question)

    # Check if the extracted answer is correct
    correct = is_correct(answer, truth)
    correct_count += int(correct)

    results.append([
        question,
        truth,
        answer,
        f"{conf:.2f}",
        f"{duration:.2f}",
        "Y" if correct else "N"
    ])

# Print results table
headers = ["Question", "Ground Truth", "Model Answer", "Confidence", "Time (s)", "Correct"]
print("\nDistilBERT Evaluation (Extractive QA)\n")
print(tabulate(results, headers=headers, tablefmt="grid"))

accuracy = correct_count / len(test_data)
print(f"\nOverall Accuracy: {accuracy:.2%}")


Loading DistilBERT model for Question Answering... (this will be fast)


Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cuda:0


Model loaded.

DistilBERT Evaluation (Extractive QA)

+------------------------------------------------------------------------+-----------------+-------------------------------------------------------+--------------+------------+-----------+
| Question                                                               | Ground Truth    | Model Answer                                          |   Confidence |   Time (s) | Correct   |
+========================================================================+=================+=======================================================+==============+============+===========+
| What was the assets current assets cash and cash equivalents in 2023?  | $1,481 million. | , and non-current investments: $1,530 million in 2023 |            0 |       0.02 | N         |
+------------------------------------------------------------------------+-----------------+-------------------------------------------------------+--------------+------------+-----------+
|

Fine Tune

In [ ]:
import logging
from torch.utils.data import Dataset
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    Trainer,
    TrainingArguments,
    DataCollatorForLanguageModeling,
    BitsAndBytesConfig,
)
from torch.utils.data import Dataset
from accelerate import Accelerator

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
logger = logging.getLogger(__name__)

In [ ]:
import json
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

file_path = '/content/drive/MyDrive/qa/financial_qa_pairs.json'
finetune_txt_path = "/content/drive/MyDrive/qa/finetuning_data.txt"

def prepare_finetuning_data(file_path, output_txt_path):
    """Converts the Q&A JSON file into a single text file for fine-tuning."""
    logger.info(f"Preparing data from {file_path} for fine-tuning.")
    try:
        with open(file_path, 'r') as f:
            data = json.load(f)
            logger.info("JSON file loaded successfully!")

        ''' with open(output_txt_path, 'w', encoding='utf-8') as outfile:
            for item in data:
                # Format each Q&A pair as a single line
                outfile.write(f"Question: {item['question']}\nAnswer: {item['answer']}\n\n") '''
        logger.info(f"Data successfully written to {output_txt_path}")
        return True
    except json.JSONDecodeError:
        logger.error(f"Error: The file at {file_path} is not a valid JSON file.")
        return False
    except Exception as e:
        logger.error(f"An error occurred during data preparation: {e}")
        return False

Mounted at /content/drive/


In [ ]:
import logging
from peft import LoraConfig, get_peft_model # Import PEFT modules
from torch.utils.data import Dataset
import torch
import os
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    Trainer,
    TrainingArguments,
    DataCollatorForLanguageModeling,
)
from accelerate import Accelerator
# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
logger = logging.getLogger(__name__)

# A more robust custom Dataset class that treats each line as an example
class QADataset(Dataset):
    def __init__(self, tokenizer, file_path, block_size):
        self.examples = []
        logger.info(f"Loading and tokenizing data from {file_path}")

        # Ensure the file exists before attempting to open it
        if not os.path.exists(file_path):
            logger.error(f"File not found: {file_path}. Please check the path.")
            return

        with open(file_path, "r", encoding="utf-8") as f:
            lines = f.readlines()

        # Tokenize each line (Q&A pair) individually
        for line in lines:
            if line.strip(): # Ensure the line is not empty
                # Using the tokenizer's __call__ method for more control over tokenization
                tokenized_line = tokenizer(
                    line.strip(),
                    add_special_tokens=True,
                    truncation=True,
                    max_length=block_size,
                    return_tensors="pt"
                )["input_ids"].squeeze(0)
                self.examples.append(tokenized_line)

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, i):
        return self.examples[i]

def fine_tune_model(data_path, model_name='gpt2', output_dir='./fine_tuned_gpt2'):
    """Fine-tunes a generative language model on the prepared Q&A dataset."""
    logger.info(f"Starting fine-tuning for model '{model_name}'.")

    # Initialize tokenizer and model with GPT-2-specific settings
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    # Load the base model. GPT-2 is a decoder-only model compatible with AutoModelForCausalLM
    logger.info("Loading GPT-2 model...")
    model = AutoModelForCausalLM.from_pretrained(
        model_name
    )

    # Set pad_token_id for GPT-2
    tokenizer.pad_token = tokenizer.eos_token
    model.config.pad_token_id = tokenizer.eos_token_id

    # Configure PEFT with Lora (Low-Rank Adaptation)
    logger.info("Setting up PEFT adapter with LoraConfig...")
    peft_config = LoraConfig(
        r=16, # Rank of the update matrices
        lora_alpha=32, # Scaling factor
        target_modules=["c_attn", "c_proj"], # Common layers for GPT-2
        lora_dropout=0.05,
        bias="none",
        task_type="CAUSAL_LM"
    )

    # Wrap the model with the PEFT adapter
    model = get_peft_model(model, peft_config)

    # Print a summary of the trainable parameters
    model.print_trainable_parameters()

    # Load dataset
    train_dataset = QADataset(
        tokenizer=tokenizer,
        file_path=data_path,
        block_size=512
    )

    # Check if dataset is empty before proceeding
    if len(train_dataset) == 0:
        logger.error("The training dataset is empty. Please check the content of the source file.")
        return

    # Data collator handles padding and creating batches
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=False
    )

    # Define training arguments
    learning_rate = 2e-4 # A common learning rate for fine-tuning with LoRA

    training_args = TrainingArguments(
        output_dir=output_dir,
        overwrite_output_dir=True,
        num_train_epochs=3,
        learning_rate=learning_rate,
        per_device_train_batch_size=2, # Reduced batch size to manage memory
        gradient_accumulation_steps=4, # Use gradient accumulation to simulate a larger batch size
        save_steps=1000,
        save_total_limit=2,
        logging_steps=50,
        report_to="none" # Disable logging to Weights & Biases
    )

    # Log hyperparameters and compute setup
    logger.info(f"--- Fine-Tuning Hyperparameters ---")
    logger.info(f"Learning Rate: {training_args.learning_rate}")
    logger.info(f"Batch Size (per device): {training_args.per_device_train_batch_size}")
    logger.info(f"Number of Epochs: {training_args.num_train_epochs}")
    logger.info(f"Compute Setup: GPU (CUDA)")
    logger.info("------------------------------------")

    # Initialize Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        data_collator=data_collator,
        train_dataset=train_dataset,
    )

    # Start training
    logger.info("Training started...")
    trainer.train()
    logger.info("Training finished.")

    # Save the fine-tuned model
    trainer.save_model(output_dir)
    tokenizer.save_pretrained(output_dir)
    logger.info(f"Fine-tuned model saved to {output_dir}")
def main():
    """Main function to run the fine-tuning pipeline."""
    # Placeholder for mounting Google Drive or other data fetching
    # This part depends on your specific environment.
    # drive.mount('/content/drive')

    # Update the path to point to your Google Drive location
    qa_json_path = "/content/drive/MyDrive/qa/financial_qa_pairs.json"
    finetune_txt_path = "/content/drive/MyDrive/qa/finetuning_data.txt"

    # You will need to implement a data preparation function here
    # prepare_finetuning_data(qa_json_path, finetune_txt_path)
    # or ensure your data file is at the specified path.

    # Step 2: Fine-tune the model
    fine_tune_model(finetune_txt_path)

if __name__ == "__main__":
    if torch.cuda.is_available():
        main()
    else:
        logger.error("A CUDA-enabled GPU is required for fine-tuning models. Please check your runtime environment.")




/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/layer.py:2156: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


trainable params: 1,622,016 || all params: 126,061,824 || trainable%: 1.2867


We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.
You may ignore this warning if your `pad_token_id` (50256) is identical to the `bos_token_id` (50256), `eos_token_id` (50256), or the `sep_token_id` (None), and your input is not padded.
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
50,3.156700
100,2.092100




# **> Distl bert pre trained Evaluation**



In [ ]:
import time
import torch
import re
from transformers import pipeline, DistilBertTokenizerFast, DistilBertForQuestionAnswering
from tabulate import tabulate
import logging

# Set up logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.StreamHandler(),
        logging.FileHandler('evaluation.log')
    ]
)
logger = logging.getLogger(__name__)

# -------------------------
# 1. Load DistilBERT Model and DistilGPT2 for Value Extraction
# -------------------------
MODEL_NAME = "distilbert-base-uncased-distilled-squad"
print(f"Loading {MODEL_NAME}... (this may take a while)")
tokenizer = DistilBertTokenizerFast.from_pretrained(MODEL_NAME)
model = DistilBertForQuestionAnswering.from_pretrained(MODEL_NAME)

# Initialize DistilGPT2 for prompt-based value extraction
llm_pipeline = pipeline("text-generation", model="distilgpt2", tokenizer="distilgpt2", device=0 if torch.cuda.is_available() else -1)
llm_pipeline.tokenizer.pad_token_id = llm_pipeline.tokenizer.eos_token_id  # Fix padding token warning

# -------------------------
# 2. Define Test Q&A Pairs
# -------------------------
test_data = [
    {
        "question": "What was the assets current assets cash and cash equivalents in 2023?",
        "answer": "For the year 2023, the assets current assets cash and cash equivalents was $1,481 million.",
        "context": "For the year 2023, the assets current assets cash and cash equivalents was $1,481 million. For the year 2022, it was $2,305 million."
    },
    {
        "question": "What was the assets current assets cash and cash equivalents in 2022?",
        "answer": "For the year 2022, the assets current assets cash and cash equivalents was $2,305 million.",
        "context": "For the year 2023, the assets current assets cash and cash equivalents was $1,481 million. For the year 2022, it was $2,305 million."
    },
    {
        "question": "What was the current investments trade receivables in 2023?",
        "answer": "For the year 2023, the current investments trade receivables was $3,094 million.",
        "context": "For the year 2023, the current investments trade receivables was $3,094 million. For the year 2022, it was $2,995 million."
    },
    {
        "question": "What was the current investments trade receivables in 2022?",
        "answer": "For the year 2022, the current investments trade receivables was $2,995 million.",
        "context": "For the year 2023, the current investments trade receivables was $3,094 million. For the year 2022, it was $2,995 million."
    },
    {
        "question": "What was the unbilled revenues in 2023?",
        "answer": "For the year 2023, the unbilled revenues was $1,861 million.",
        "context": "For the year 2023, the unbilled revenues was $1,861 million. For the year 2022, it was $1,526 million."
    },
    {
        "question": "What was the unbilled revenues in 2022?",
        "answer": "For the year 2022, the unbilled revenues was $1,526 million.",
        "context": "For the year 2023, the unbilled revenues was $1,861 million. For the year 2022, it was $1,526 million."
    },
    {
        "question": "What was the prepayments and other current assets in 2023?",
        "answer": "For the year 2023, the prepayments and other current assets was $1,336 million.",
        "context": "For the year 2023, the prepayments and other current assets was $1,336 million. For the year 2022, it was $1,133 million."
    },
    {
        "question": "What was the prepayments and other current assets in 2022?",
        "answer": "For the year 2022, the prepayments and other current assets was $1,133 million.",
        "context": "For the year 2023, the prepayments and other current assets was $1,336 million. For the year 2022, it was $1,133 million."
    },
    {
        "question": "What was the income tax assets derivative financial instruments total current assets in 2023?",
        "answer": "For the year 2023, the income tax assets derivative financial instruments total current assets was $8,626 million.",
        "context": "For the year 2023, the income tax assets derivative financial instruments total current assets was $8,626 million. For the year 2022, it was $8,865 million."
    },
    {
        "question": "What was the income tax assets derivative financial instruments total current assets in 2022?",
        "answer": "For the year 2022, the income tax assets derivative financial instruments total current assets was $8,865 million.",
        "context": "For the year 2023, the income tax assets derivative financial instruments total current assets was $8,626 million. For the year 2022, it was $8,865 million."
    },
    {
        "question": "What was the non-current assets property, plant and equipment in 2023?",
        "answer": "For the year 2023, the non-current assets property, plant and equipment was $1,679 million.",
        "context": "For the year 2023, the non-current assets property, plant and equipment was $1,679 million. For the year 2022, it was $1,793 million."
    },
    {
        "question": "What was the non-current assets property, plant and equipment in 2022?",
        "answer": "For the year 2022, the non-current assets property, plant and equipment was $1,793 million.",
        "context": "For the year 2023, the non-current assets property, plant and equipment was $1,679 million. For the year 2022, it was $1,793 million."
    },
    {
        "question": "What was the right-of-use assets goodwill intangible assets non-current investments in 2023?",
        "answer": "For the year 2023, the right-of-use assets goodwill intangible assets non-current investments was $1,530 million.",
        "context": "For the year 2023, the right-of-use assets goodwill intangible assets non-current investments was $1,530 million."
    }
]

# -------------------------
# 3. Helper Functions
# -------------------------
def extract_value_with_prompt(answer, llm_pipeline):
    """Extract numerical value using prompt-based approach."""
    if not answer:
        return None
    # Fallback regex for simple integers
    match = re.match(r'^-?\d+$', answer)
    if match:
        return match.group(0)
    # Prompt-based extraction
    prompt = (
        f"Extract the main numerical value from the text below and return only the clean integer as a string, "
        f"removing commas, dollar signs, parentheses, and words like 'million'. "
        f"If no valid integer is found, return 'None'. Do not include any extra text or explanations. "
        f"Examples: '$1,234 million' -> '1234', '1234345' -> '1234345', '(1,234)' -> '1234', '-1234' -> '-1234', "
        f"'$18,212 million' -> '18212', '18,212.50 million' -> '18212', 'no number' -> 'None'. "
        f"Text: '{answer}'"
    )
    try:
        result = llm_pipeline(prompt, max_new_tokens=20, num_return_sequences=1)[0]['generated_text']
        number_match = re.search(r'-?\d+', result)
        if number_match:
            cleaned_result = number_match.group(0)
            if cleaned_result.lstrip('-').isdigit():
                return cleaned_result
        return None
    except Exception as e:
        logger.error(f"Error in prompt-based extraction for '{answer}': {e}")
        return None

def get_model_answer(question, context, qa_pipeline):
    """Generate answer, confidence, and time using DistilBERT QA pipeline."""
    start = time.time()
    result = qa_pipeline(question=question, context=context)
    end = time.time()
    return result['answer'], result['score'], end - start

def is_correct(pred_value, truth_value):
    """Check if predicted numerical value matches ground truth value."""
    return pred_value == truth_value

# -------------------------
# 4. Run Evaluation
# -------------------------
qa_pipeline = pipeline(
    "question-answering",
    model=model,
    tokenizer=tokenizer,
    device=0 if torch.cuda.is_available() else -1
)

results = []
correct_count = 0

for item in test_data[:10]:  # Limit to 10 for benchmarking
    question = item["question"]
    truth = item["answer"]
    context = item["context"]

    # Get model answer
    answer, conf, duration = get_model_answer(question, context, qa_pipeline)

    # Extract numerical values for comparison
    pred_value = extract_value_with_prompt(answer, llm_pipeline)
    truth_value = extract_value_with_prompt(truth, llm_pipeline)

    # Check correctness
    correct = is_correct(pred_value, truth_value)
    correct_count += int(correct)

    # Log results
    results.append([
        question[:60] + ("..." if len(question) > 60 else ""),
        truth[:60] + ("..." if len(truth) > 60 else ""),
        answer[:60] + ("..." if len(answer) > 60 else ""),
        pred_value if pred_value else "None",
        truth_value if truth_value else "None",
        f"{conf:.2f}",
        f"{duration:.2f}",
        "Y" if correct else "N"
    ])

# -------------------------
# 5. Print Results Table
# -------------------------
headers = ["Question", "Ground Truth", "Model Answer", "Pred Value", "Truth Value", "Confidence", "Time (s)", "Correct"]
print("\nBaseline Evaluation (DistilBERT Pre-trained)\n")
print(tabulate(results, headers=headers, tablefmt="grid"))

accuracy = correct_count / len(results)
print(f"\nOverall Accuracy: {accuracy:.2%}")

Loading distilbert-base-uncased-distilled-squad... (this may take a while)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cuda:0
Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 fo


Baseline Evaluation (DistilBERT Pre-trained)

+-----------------------------------------------------------------+-----------------------------------------------------------------+----------------+--------------+---------------+--------------+------------+-----------+
| Question                                                        | Ground Truth                                                    | Model Answer   |   Pred Value |   Truth Value |   Confidence |   Time (s) | Correct   |
+=================================================================+=================================================================+================+==============+===============+==============+============+===========+
| What was the assets current assets cash and cash equivalents... | For the year 2023, the assets current assets cash and cash e... | $1,481 million |            1 |             1 |         0.98 |       0.67 | Y         |
+----------------------------------------------------------------



# **> Falcon Evaluation base model with Financial data set**



In [ ]:
import time
import torch
import re
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from torch.nn.functional import softmax
from tabulate import tabulate
import logging

# Set up logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.StreamHandler(),
        logging.FileHandler('evaluation.log')
    ]
)
logger = logging.getLogger(__name__)

# -------------------------
# 1. Load Falcon-7B Model and DistilGPT2 for Value Extraction
# -------------------------
MODEL_NAME = "tiiuae/falcon-7b-instruct"
print(f"Loading {MODEL_NAME}... (this may take a while)")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.bfloat16,
    device_map="auto"  # Automatically place model on GPU if available
)

# Initialize DistilGPT2 for prompt-based value extraction
llm_pipeline = pipeline("text-generation", model="distilgpt2", tokenizer="distilgpt2", device=0 if torch.cuda.is_available() else -1)
llm_pipeline.tokenizer.pad_token_id = llm_pipeline.tokenizer.eos_token_id  # Fix padding token warning

# -------------------------
# 2. Define Test Q&A Pairs
# -------------------------
test_data = [
    {
        "question": "What was the assets current assets cash and cash equivalents in 2023?",
        "answer": "For the year 2023, the assets current assets cash and cash equivalents was $1,481 million."
    },
	 {
        "question": "What was the assets current assets cash and cash equivalents in 2024?",
        "answer": "For the year 2024, the assets current assets cash and cash equivalents was $1,773 million."
    },
    {
        "question": "What was the current investments trade receivables in 2023?",
        "answer": "For the year 2023, the current investments trade receivables was $3,094 million."
    },
	{
        "question": "What was the current investments 1,548 trade receivables in 2024?",
        "answer": "For the year 2024, the current investments 1,548 trade receivables was $3,620 million."
    },
    {
        "question": "What was the unbilled revenues in 2023?",
        "answer": "For the year 2023, the unbilled revenues was $1,861 million."
    },
	{
        "question": "What was the unbilled revenues in 2024?",
        "answer": "For the year 2024, the unbilled revenues was $1,531 million."
    },
    {
        "question": "What was the prepayments and other current assets in 2023?",
        "answer": "For the year 2023, the prepayments and other current assets was $1,336 million."
    },

    {
        "question": "What was the prepayments and other current assets in 2024?",
        "answer": "For the year 2024, the prepayments and other current assets was $1,473 million."
    },
	{
        "question": "What was the total assets in 2023?",
        "answer": "For the year 2023, the total assets was $15,312 million."
    },
	{
        "question": "What was the total assets in 2024?",
        "answer": "For the year 2024, the total assets was $16,523 million."
    }
]

# -------------------------
# 3. Helper Functions
# -------------------------
def extract_value_with_prompt(answer, llm_pipeline):
    """Extract numerical value using prompt-based approach."""
    if not answer:
        return None
    # Fallback regex for simple integers and formatted numbers
    match = re.match(r'^-?\d+$|^-?\d{1,3}(?:,\d{3})*(?:\.\d+)?$|^\(-?\d{1,3}(?:,\d{3})*\)$', answer)
    if match:
        value = match.group(0).replace(',', '').replace('(', '').replace(')', '')
        if value.lstrip('-').isdigit():
            return value
    # Prompt-based extraction
    prompt = (
        f"Extract the main numerical value from the text below and return only the clean integer as a string, "
        f"removing commas, dollar signs, parentheses, and words like 'million'. "
        f"If no valid integer is found, return 'None'. Do not include any extra text or explanations. "
        f"Examples: '$1,234 million' -> '1234', '1234345' -> '1234345', '(1,234)' -> '1234', '-1234' -> '-1234', "
        f"'$18,212 million' -> '18212', '18,212.50 million' -> '18212', 'no number' -> 'None'. "
        f"Text: '{answer}'"
    )
    try:
        result = llm_pipeline(prompt, max_new_tokens=20, num_return_sequences=1)[0]['generated_text']
        number_match = re.search(r'-?\d+', result)
        if number_match:
            cleaned_result = number_match.group(0)
            if cleaned_result.lstrip('-').isdigit():
                return cleaned_result
        return None
    except Exception as e:
        logger.error(f"Error in prompt-based extraction for '{answer}': {e}")
        return None

def get_model_answer(question, max_new_tokens=50):
    """Generate answer, confidence, and time using Falcon-7B."""
    inputs = tokenizer(question, return_tensors="pt").to(model.device)

    start = time.time()
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        return_dict_in_generate=True,
        output_scores=True
    )
    end = time.time()

    # Decode model output
    answer = tokenizer.decode(outputs.sequences[0], skip_special_tokens=True)
    answer = answer.replace(question, "").strip()  # Clean echo

    # Confidence (average max probability across generated tokens)
    scores = torch.stack(outputs.scores, dim=1)  # (batch, seq_len, vocab_size)
    probs = softmax(scores, dim=-1)
    max_probs = probs.max(-1).values.mean().item()

    return answer, max_probs, end - start

def is_correct(pred_value, truth_value):
    """Check if predicted numerical value matches ground truth value."""
    return pred_value == truth_value

# -------------------------
# 4. Run Evaluation
# -------------------------
results = []
correct_count = 0

for item in test_data[:10]:  # Limit to 10 for benchmarking
    question = item["question"]
    truth = item["answer"]

    # Get model answer
    answer, conf, duration = get_model_answer(question)

    # Extract numerical values for comparison
    pred_value = extract_value_with_prompt(answer, llm_pipeline)
    truth_value = extract_value_with_prompt(truth, llm_pipeline)

    # Check correctness
    correct = is_correct(pred_value, truth_value)
    correct_count += int(correct)

    # Log results
    results.append([
        question[:60] + ("..." if len(question) > 60 else ""),
        truth[:60] + ("..." if len(truth) > 60 else ""),
        answer[:60] + ("..." if len(answer) > 60 else ""),
        pred_value if pred_value else "None",
        truth_value if truth_value else "None",
        f"{conf:.2f}",
        f"{duration:.2f}",
        "Y" if correct else "N"
    ])

# -------------------------
# 5. Print Results Table
# -------------------------
headers = ["Question", "Ground Truth", "Model Answer", "Pred Value", "Truth Value", "Confidence", "Time (s)", "Correct"]
print("\nBaseline Evaluation (Falcon-7B)\n")
print(tabulate(results, headers=headers, tablefmt="grid"))

accuracy = correct_count / len(results)
print(f"\nOverall Accuracy: {accuracy:.2%}")

Loading tiiuae/falcon-7b-instruct... (this may take a while)


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/4.48G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for ope


Baseline Evaluation (Falcon-7B)

+-----------------------------------------------------------------+-----------------------------------------------------------------+-----------------------------------------------------------------+--------------+---------------+--------------+------------+-----------+
| Question                                                        | Ground Truth                                                    | Model Answer                                                    |   Pred Value |   Truth Value |   Confidence |   Time (s) | Correct   |
+=================================================================+=================================================================+=================================================================+==============+===============+==============+============+===========+
| What was the assets current assets cash and cash equivalents... | For the year 2023, the assets current assets cash and cash e... | I'm sorry, I cannot

In [ ]:
!pip install tabulate
!pip install rank_bm25
!pip install bitsandbytes
!pip install transformers peft accelerate torch


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import time
from tabulate import tabulate

model_name = "tiiuae/falcon-7b-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", torch_dtype=torch.bfloat16)

def ask(question, max_new_tokens=150):
    if not isinstance(question, str):
        question = str(question)
    inputs = tokenizer(question, return_tensors="pt").to(model.device)
    start = time.time()
    outputs = model.generate(**inputs, max_new_tokens=max_new_tokens)
    latency = time.time() - start
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer, latency

test_questions = [
     {
        "question": "What was the assets current assets cash and cash equivalents in 2023?",
        "answer": "For the year 2023, the assets current assets cash and cash equivalents was $1,481 million."
    },
	 {
        "question": "What was the assets current assets cash and cash equivalents in 2024?",
        "answer": "For the year 2024, the assets current assets cash and cash equivalents was $1,773 million."
    },
    {
        "question": "What was the current investments trade receivables in 2023?",
        "answer": "For the year 2023, the current investments trade receivables was $3,094 million."
    },
	{
        "question": "What was the current investments 1,548 trade receivables in 2024?",
        "answer": "For the year 2024, the current investments 1,548 trade receivables was $3,620 million."
    },
    {
        "question": "What was the unbilled revenues in 2023?",
        "answer": "For the year 2023, the unbilled revenues was $1,861 million."
    },
	{
        "question": "What was the unbilled revenues in 2024?",
        "answer": "For the year 2024, the unbilled revenues was $1,531 million."
    },
    {
        "question": "What was the prepayments and other current assets in 2023?",
        "answer": "For the year 2023, the prepayments and other current assets was $1,336 million."
    },

    {
        "question": "What was the prepayments and other current assets in 2024?",
        "answer": "For the year 2024, the prepayments and other current assets was $1,473 million."
    },
	{
        "question": "What was the total assets in 2023?",
        "answer": "For the year 2023, the total assets was $15,312 million."
    },
	{
        "question": "What was the total assets in 2024?",
        "answer": "For the year 2024, the total assets was $16,523 million."
    }
]

results = []
for i, item in enumerate(test_questions[:10]):  # test first 10 Qs for baseline
    q, gold = item["question"], item["answer"]
    model_ans, latency = ask(q)
    results.append({
        "question": q,
        "expected_answer": gold,
        "model_answer": model_ans,
        "accuracy": int(gold.lower() in model_ans.lower()),  # simple string match
        "latency_sec": round(latency, 2)
    })
    print(f"[{i+1}] Q: {q}\nModel: {model_ans}\nGold: {gold}\n")
# -------------------------
# 5. Print Results Table
# -------------------------
headers = ["Question", "Ground Truth", "Model Answer", "Pred Value", "Truth Value", "Confidence", "Time (s)", "Correct"]
print("\nBaseline Evaluation (Falcon-7B)\n")
print(tabulate(results, headers=headers, tablefmt="grid"))

accuracy = correct_count / len(results)
print(f"\nOverall Accuracy: {accuracy:.2%}")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


[1] Q: What was the assets current assets cash and cash equivalents in 2023?
Model: What was the assets current assets cash and cash equivalents in 2023?
I'm sorry, I cannot provide a specific answer without additional context. Can you please provide more information?
Gold: For the year 2023, the assets current assets cash and cash equivalents was $1,481 million.



Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


[2] Q: What was the assets current assets cash and cash equivalents in 2024?
Model: What was the assets current assets cash and cash equivalents in 2024?
I'm sorry, I cannot provide a specific answer without additional context. Can you please provide more information?
Gold: For the year 2024, the assets current assets cash and cash equivalents was $1,773 million.



Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


[3] Q: What was the current investments trade receivables in 2023?
Model: What was the current investments trade receivables in 2023?
I'm sorry, I cannot provide a specific answer without additional context. Can you please provide more information?
Gold: For the year 2023, the current investments trade receivables was $3,094 million.



KeyboardInterrupt: 

Loading DistilBERT model for Question Answering... (this will be fast)


Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cuda:0


Model loaded.

DistilBERT Evaluation (Extractive QA)

+------------------------------------------------------------------------+-----------------+-------------------------------------------------------+--------------+------------+-----------+
| Question                                                               | Ground Truth    | Model Answer                                          |   Confidence |   Time (s) | Correct   |
+========================================================================+=================+=======================================================+==============+============+===========+
| What was the assets current assets cash and cash equivalents in 2023?  | $1,481 million. | , and non-current investments: $1,530 million in 2023 |            0 |       0.02 | N         |
+------------------------------------------------------------------------+-----------------+-------------------------------------------------------+--------------+------------+-----------+
|



# **> Distl Bert**



In [ ]:
import time
import torch
from transformers import pipeline, AutoTokenizer
from tabulate import tabulate
import json

# Define the model name for DistilBERT
MODEL_NAME = "distilbert-base-uncased"

# For extractive QA models, the context needs to be well-structured
# to help the model find the correct answer span. A single, long
# string of disjointed facts is difficult for the model to process.
# We will use a more structured context here.
CONTEXT = """
The company's financial results for the years 2023 and 2022 are as follows:
- Assets:
- Cash and cash equivalents: $1,481 million in 2023 and $2,305 million in 2022.
- Current investments trade receivables: $3,094 million in 2023 and $2,995 million in 2022.
- Unbilled revenues: $1,861 million in 2023 and $1,526 million in 2022.
- Prepayments and other current assets: $1,336 million in 2023 and $1,133 million in 2022.
- Income tax assets, derivative financial instruments, and total current assets: $8,626 million in 2023 and $8,865 million in 2022.
- Non-current assets property, plant and equipment: $1,679 million in 2023 and $1,793 million in 2022.
- Right-of-use assets, goodwill, intangible assets, and non-current investments: $1,530 million in 2023.
"""

# Load the question-answering pipeline
print("Loading DistilBERT model for Question Answering... (this will be fast)")
qa_pipeline = pipeline(
    "question-answering",
    model=MODEL_NAME,
    tokenizer=MODEL_NAME,
    device=0 if torch.cuda.is_available() else -1
)
print("Model loaded.")

# Define your test Q&A pairs
test_data = [
    {
        "question": "What was the assets current assets cash and cash equivalents in 2023?",
        "answer": "For the year 2023, the assets current assets cash and cash equivalents was $1,481 million."
    },
    {
        "question": "What was the assets current assets cash and cash equivalents in 2022?",
        "answer": "For the year 2022, the assets current assets cash and cash equivalents was $2,305 million."
    },
    {
        "question": "What was the current investments trade receivables in 2023?",
        "answer": "For the year 2023, the current investments trade receivables was $3,094 million."
    },
    {
        "question": "What was the current investments trade receivables in 2022?",
        "answer": "For the year 2022, the current investments trade receivables was $2,995 million."
    },
    {
        "question": "What was the unbilled revenues in 2023?",
        "answer": "For the year 2023, the unbilled revenues was $1,861 million."
    },
    {
        "question": "What was the unbilled revenues in 2022?",
        "answer": "For the year 2022, the unbilled revenues was $1,526 million."
    },
    {
        "question": "What was the prepayments and other current assets in 2023?",
        "answer": "For the year 2023, the prepayments and other current assets was $1,336 million."
    },
    {
        "question": "What was the prepayments and other current assets in 2022?",
        "answer": "For the year 2022, the prepayments and other current assets was $1,133 million."
    },
    {
        "question": "What was the income tax assets derivative financial instruments total current assets in 2023?",
        "answer": "For the year 2023, the income tax assets derivative financial instruments total current assets was $8,626 million."
    },
    {
        "question": "What was the income tax assets derivative financial instruments total current assets in 2022?",
        "answer": "For the year 2022, the income tax assets derivative financial instruments total current assets was $8,865 million."
    },
    {
        "question": "What was the non-current assets property, plant and equipment in 2023?",
        "answer": "For the year 2023, the non-current assets property, plant and equipment was $1,679 million."
    },
    {
        "question": "What was the non-current assets property, plant and equipment in 2022?",
        "answer": "For the year 2022, the non-current assets property, plant and equipment was $1,793 million."
    },
    {
        "question": "What was the right-of-use assets goodwill intangible assets non-current investments in 2023?",
        "answer": "For the year 2023, the right-of-use assets goodwill intangible assets non-current investments was $1,530 million."
    }
]

# Helper function to get model answer with confidence
def get_model_answer(question):
    """
    Generate answer from the context using the QA pipeline.
    """
    start = time.time()
    result = qa_pipeline(question=question, context=CONTEXT)
    end = time.time()

    answer = result['answer']
    confidence = result['score']
    duration = end - start

    return answer, confidence, duration

def is_correct(pred, truth):
    """Check if ground truth string is in prediction."""
    return truth.lower() in pred.lower()

# Run evaluation
results = []
correct_count = 0

for item in test_data:
    question = item["question"]
    truth = item["answer"]
    answer, conf, duration = get_model_answer(question)

    # Check if the extracted answer is correct
    correct = is_correct(answer, truth)
    correct_count += int(correct)

    results.append([
        question,
        truth,
        answer,
        f"{conf:.2f}",
        f"{duration:.2f}",
        "Y" if correct else "N"
    ])

# Print results table
headers = ["Question", "Ground Truth", "Model Answer", "Confidence", "Time (s)", "Correct"]
print("\nDistilBERT Evaluation (Extractive QA)\n")
print(tabulate(results, headers=headers, tablefmt="grid"))

accuracy = correct_count / len(test_data)
print(f"\nOverall Accuracy: {accuracy:.2%}")


Loading DistilBERT model for Question Answering... (this will be fast)


Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cuda:0


Model loaded.

DistilBERT Evaluation (Extractive QA)

+------------------------------------------------------------------------+-----------------+-------------------------------------------------------+--------------+------------+-----------+
| Question                                                               | Ground Truth    | Model Answer                                          |   Confidence |   Time (s) | Correct   |
+========================================================================+=================+=======================================================+==============+============+===========+
| What was the assets current assets cash and cash equivalents in 2023?  | $1,481 million. | , and non-current investments: $1,530 million in 2023 |            0 |       0.02 | N         |
+------------------------------------------------------------------------+-----------------+-------------------------------------------------------+--------------+------------+-----------+
|

In [7]:
import json
import logging
import os
import torch
import re
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForCausalLM
import time
from tabulate import tabulate
from typing import Dict, List, Tuple, Optional

# Attempt to import and mount Google Drive, handle error if not in Colab
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    base_dir = '/content/drive/My Drive/qa'  # Adjust if your folder structure is different
    IN_COLAB = True
except ImportError:
    base_dir = '.' # Default to current directory if not in Colab
    IN_COLAB = False


# -------------------------
# 1. Configuration
# -------------------------
# Using DistilGPT-2 as it's a small, open-source generative model suitable for this task.
MODEL_NAME = "distilgpt2"

# ----------------------------
# Utilities: scoring functions
# ----------------------------

def normalize_text(s: str) -> str:
    return re.sub(r"\s+", " ", s.strip().lower())

def exact_match(pred: str, gold: str) -> int:
    return int(normalize_text(gold) in normalize_text(pred))

def f1_score(pred: str, gold: str) -> float:
    p_tokens = normalize_text(pred).split()
    g_tokens = normalize_text(gold).split()
    if not p_tokens or not g_tokens:
        return 0.0
    common = 0
    counts = {}
    for t in g_tokens:
        counts[t] = counts.get(t, 0) + 1
    for t in p_tokens:
        if counts.get(t, 0) > 0:
            common += 1
            counts[t] -= 1
    if common == 0:
        return 0.0
    precision = common / len(p_tokens)
    recall = common / len(g_tokens)
    return 2 * precision * recall / (precision + recall)

def extract_numbers(text: str) -> List[float]:
    # capture numbers like 4.13, 18.2, 1,000, 4.13 billion (we'll strip units)
    # we return plain floats; commas removed.
    nums = []
    # Replace commas in numbers
    def _clean_num(n):
        return n.replace(",", "")
    for m in re.finditer(r"(?<!\w)(-?\d{1,3}(?:,\d{3})*(?:\.\d+)?|-?\d+(?:\.\d+)?)(?!\w)", text):
        try:
            nums.append(float(_clean_num(m.group(0))))
        except:
            pass
    return nums

def numeric_match(pred: str, gold: str, rel_tol: float = 0.05) -> int:
    """Return 1 if any number in pred is within rel_tol of any number in gold; else 0.
    rel_tol=0.05 means 5% tolerance."""
    p_nums = extract_numbers(pred)
    g_nums = extract_numbers(gold)
    if not p_nums or not g_nums:
        return 0
    for g in g_nums:
        for p in p_nums:
            if g == 0:
                if abs(p - g) < 1e-9:
                    return 1
            else:
                if abs(p - g) / abs(g) <= rel_tol:
                    return 1
    return 0
def build_prompt(question: str, context: Optional[str], style: str) -> str:
    if style == "rag" and context and context.strip():
        return f"Use the context to answer the question.\nContext: {context.strip()}\nQuestion: {question.strip()}"
    else:
        return f"Answer the question concisely.\nQuestion: {question.strip()}"

# -------------------------
# 2. Custom Dataset Class
# -------------------------
class QADataset(Dataset):
    """Custom dataset for loading Q&A text file."""
    def __init__(self, tokenizer, file_path, block_size):
        print(f"Loading and tokenizing data from {file_path}")
        with open(file_path, 'r', encoding='utf-8') as f:
            text = f.read()

        tokenized_text = tokenizer.encode(text)

        self.examples = []
        for i in range(0, len(tokenized_text) - block_size + 1, block_size):
            self.examples.append(torch.tensor(tokenized_text[i:i + block_size], dtype=torch.long))
        print(f"Created {len(self.examples)} examples.")

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, i):
        return self.examples[i]

# -------------------------
# 3. Data Preparation
# -------------------------
def prepare_finetuning_data(qa_file_path, output_txt_path):
    """Converts the Q&A JSON file into a single text file for fine-tuning."""
    print(f"Preparing data from {qa_file_path} for fine-tuning.")
    try:
        with open(qa_file_path, 'r', encoding='utf-8') as f:
            qa_pairs = json.load(f)

        formatted_text = ""
        for pair in qa_pairs:
            instruction = "You are a financial analyst bot. Your task is to answer questions based on the provided financial data."
            input_text = pair.get('question') or pair.get('input')
            output_text = pair.get('answer') or pair.get('output')

            if not input_text or not output_text:
                continue

            formatted_text += f"### Instruction:\n{instruction}\n\n### Input:\n{input_text}\n\n### Response:\n{output_text}\n\n"

        with open(output_txt_path, 'w', encoding='utf-8') as f:
            f.write(formatted_text)

        print(f"Successfully created fine-tuning data at {output_txt_path}")
        return True
    except FileNotFoundError:
        logger.error(f"Error: The file was not found at {qa_file_path}")
        return False
    except Exception as e:
        logger.error(f"Failed to prepare fine-tuning data: {e}")
        return False

# -------------------------
# 4. Baseline Benchmarking
# -------------------------
def pretrained_model_evaluation(model_name, finetune_txt_path, test_questions):
    """Evaluates a pre-trained model on the fine-tuning dataset and specific test questions."""
    print(f"Starting evaluation for pre-trained model '{model_name}'.")

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    # --- Part 1: Calculate Perplexity on the dataset ---
    print("Calculating perplexity on the entire dataset...")
    try:
        dataset = QADataset(tokenizer, file_path=finetune_txt_path, block_size=128)
        if not dataset.examples:
            logger.warning("Dataset is empty. Skipping perplexity calculation.")
        else:
            dataloader = DataLoader(dataset, batch_size=8)
            model.eval()
            total_loss = 0.0
            with torch.no_grad():
                for batch in dataloader:
                    inputs = batch.to(device)
                    outputs = model(inputs, labels=inputs)
                    loss = outputs.loss
                    total_loss += loss.item()

            avg_loss = total_loss / len(dataloader)
            perplexity = torch.exp(torch.tensor(avg_loss))
            print(f"Average evaluation loss: {avg_loss:.4f}")
            print(f"Perplexity: {perplexity.item():.4f}")
    except FileNotFoundError:
        print(f"Finetuning data not found at {finetune_txt_path}. Skipping perplexity calculation.")


    # --- Part 2: Generate answers for test questions and tabulate results ---
    logger.info("\n--- Generating answers to test questions ---")
    results_data = []
    total_correct = 0
    for item in test_questions:
        question = item["question"]
        expected_answer = item["answer"]
        prompt = f"""Question: {question}
    Provided Answer: {expected_answer}
    Document Context: [Your full financial context here]...

    Extract or generate the correct ground truth value from the document that matches the question.
    If mismatched, correct it based on the document. Output only the value (e.g., '$1,481 million').
    """

        inputs = tokenizer(prompt, return_tensors="pt").to(device)
        input_ids = inputs['input_ids']
        print(f"\input_ids : {input_ids}")
        start_time = time.time()
        # Generate text with scores for confidence calculation
        outputs = model.generate(
            input_ids=input_ids,
            max_new_tokens=100,
            temperature=0.7,
            top_k=50,
            top_p=0.95,
            num_return_sequences=1,
            pad_token_id=tokenizer.eos_token_id,
            output_scores=True,
            return_dict_in_generate=True
        )
        end_time = time.time()

        generated_ids = outputs.sequences[:, input_ids.shape[-1]:]
        generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
        print(f"\generated_text : {generated_text}")
        # --- Calculate Confidence Score ---
        # Get the log probabilities of the generated tokens
        scores = torch.stack(outputs.scores, dim=1).log_softmax(-1)
        generated_token_logprobs = scores.gather(2, generated_ids[:, :, None]).squeeze(-1)

        # Calculate average probability (confidence)
        avg_prob = generated_token_logprobs.mean().exp().item() * 100

        # --- Calculate Accuracy ---
        # A simple accuracy check: does the generated answer contain the key number?
        is_correct = False
        # Extract numbers (e.g., 1,481 or 15,312) from the expected answer
        #expected_numbers = re.findall(r'\$([\d,]+)', expected_answer)
        expected_numbers = re.findall(r'\$([\d,]+)(?: million)?', expected_answer)
        if expected_numbers:
            if any(num in generated_text for num in expected_numbers):
                is_correct = True
                total_correct += 1

        accuracy_score = 1 if is_correct else 0

        results_data.append([
            question,
            expected_answer,
            generated_text,
            f"{avg_prob:.2f}%",
            accuracy_score,
            f"{end_time - start_time:.4f}"
        ])

    # Print the tabulated results
    headers = ["Question", "Expected Answer", "Generated Answer", "Confidence", "Accuracy (1/0)", "Inference Speed (s)"]
    print(tabulate(results_data, headers=headers, tablefmt="grid"))

    # Print final accuracy score
    overall_accuracy = (total_correct / len(test_questions)) * 100
    print(f"\nOverall Accuracy: {overall_accuracy:.2f}% ({total_correct}/{len(test_questions)})")


# -------------------------
# 5. Main Execution
# -------------------------
def main():
    """Main function to run the fine-tuning pipeline."""
    qa_json_path = os.path.join(base_dir, "financial_qa_pairs.json")
    finetune_txt_path = os.path.join(base_dir, "finetuning_data.txt")

    print(f"Pipeline started with QA JSON path: {qa_json_path}")

    if not os.path.exists(qa_json_path):
        print(f"The source file '{qa_json_path}' was not found.")
        return

    if not prepare_finetuning_data(qa_json_path, finetune_txt_path):
        print("Data preparation failed. Exiting.")
        return

    test_questions = [
        {"question": "What was the assets current assets cash and cash equivalents in 2023?", "answer": "For the year 2023, the assets current assets cash and cash equivalents was $1,481 million."},
        {"question": "What was the current investments trade receivables in 2023?", "answer": "For the year 2023, the current investments trade receivables was $3,094 million."},
        {"question": "What was the unbilled revenues in 2023?", "answer": "For the year 2023, the unbilled revenues was $1,861 million."},
        {"question": "What was the prepayments and other current assets in 2023?", "answer": "For the year 2023, the prepayments and other current assets was $1,336 million."},
        {"question": "What was the income tax assets derivative financial instruments total current assets in 2023?", "answer": "For the year 2023, the income tax assets derivative financial instruments total current assets was $8,626 million."},
        {"question": "What was the non-current assets property, plant and equipment in 2023?", "answer": "For the year 2023, the non-current assets property, plant and equipment was $1,679 million."},
        {"question": "What was the right-of-use assets goodwill intangible assets non-current investments in 2023?", "answer": "For the year 2023, the right-of-use assets goodwill intangible assets non-current investments was $1,530 million."},
        {"question": "What was the unbilled revenues deferred income tax assets income tax assets other non-current assets total non-current assets in 2023?", "answer": "For the year 2023, the unbilled revenues deferred income tax assets income tax assets other non-current assets total non-current assets was $6,686 million."},
        {"question": "What was the total assets in 2023?", "answer": "For the year 2023, the total assets was $15,312 million."},
        {"question": "What was the liabilities and equity current liabilities trade payables lease liabilities derivative financial instruments current income tax liabilities unearned revenues employee benefit obligations provisions other current liabilities in 2023?", "answer": "For the year 2023, the liabilities and equity current liabilities trade payables lease liabilities derivative financial instruments current income tax liabilities unearned revenues employee benefit obligations provisions other current liabilities was $2,403 million."}
    ]

    pretrained_model_evaluation(MODEL_NAME, finetune_txt_path, test_questions)


if __name__ == "__main__":
    main()


Mounted at /content/drive
Pipeline started with QA JSON path: /content/drive/My Drive/qa/financial_qa_pairs.json
Preparing data from /content/drive/My Drive/qa/financial_qa_pairs.json for fine-tuning.
Successfully created fine-tuning data at /content/drive/My Drive/qa/finetuning_data.txt
Starting evaluation for pre-trained model 'distilgpt2'.
Using device: cuda


Token indices sequence length is longer than the specified maximum sequence length for this model (3821 > 1024). Running this sequence through the model will result in indexing errors


Calculating perplexity on the entire dataset...
Loading and tokenizing data from /content/drive/My Drive/qa/finetuning_data.txt
Created 29 examples.


The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Average evaluation loss: 3.1941
Perplexity: 24.3887
\input_ids : tensor([[24361,    25,  1867,   373,   262,  6798,  1459,  6798,  5003,   290,
          5003, 40255,   287,  1160,  1954,    30,   198,   220,   220,   220,
         29750, 23998,    25,  1114,   262,   614,  1160,  1954,    11,   262,
          6798,  1459,  6798,  5003,   290,  5003, 40255,   373,   720,    16,
            11, 40271,  1510,    13,   198,   220,   220,   220, 16854, 30532,
            25,   685,  7120,  1336,  3176,  4732,   994,    60,   986,   628,
           220,   220,   220, 29677,   393,  7716,   262,  3376,  2323,  3872,
          1988,   422,   262,  3188,   326,  7466,   262,  1808,    13,   198,
           220,   220,   220,  1002, 32691, 14265,    11,  3376,   340,  1912,
           319,   262,  3188,    13, 25235,   691,   262,  1988,   357,    68,
            13,    70,  1539,   705,     3,    16,    11, 40271,  1510, 27691,
           198,   220,   220,   220,   220]], device='cuda:0')


...

\generated_text :                                                                                                     
\input_ids : tensor([[24361,    25,  1867,   373,   262, 22619,  2967, 13089, 28651,  3739,
          1687,  6798,  3739,  1687,  6798,   584,  1729,    12, 14421,  6798,
          2472,  1729,    12, 14421,  6798,   287,  1160,  1954,    30,   198,
           220,   220,   220, 29750, 23998,    25,  1114,   262,   614,  1160,
          1954,    11,   262, 22619,  2967, 13089, 28651,  3739,  1687,  6798,
          3739,  1687,  6798,   584,  1729,    12, 14421,  6798,  2472,  1729,
            12, 14421,  6798,   373,   720,    21,    11, 33808,  1510,    13,
           198,   220,   220,   220, 16854, 30532,    25,   685,  7120,  1336,
          3176,  4732,   994,    60,   986,   628,   220,   220,   220, 29677,
           393,  7716,   262,  3376,  2323,  3872,  1988,   422,   262,  3188,
           326,  7466,   262,  1808,    13,   198,   220,   220,   220,  1002,

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


\generated_text :                                                                                                     
\input_ids : tensor([[24361,    25,  1867,   373,   262,  2472,  6798,   287,  1160,  1954,
            30,   198,   220,   220,   220, 29750, 23998,    25,  1114,   262,
           614,  1160,  1954,    11,   262,  2472,  6798,   373,   720,  1314,
            11, 27970,  1510,    13,   198,   220,   220,   220, 16854, 30532,
            25,   685,  7120,  1336,  3176,  4732,   994,    60,   986,   628,
           220,   220,   220, 29677,   393,  7716,   262,  3376,  2323,  3872,
          1988,   422,   262,  3188,   326,  7466,   262,  1808,    13,   198,
           220,   220,   220,  1002, 32691, 14265,    11,  3376,   340,  1912,
           319,   262,  3188,    13, 25235,   691,   262,  1988,   357,    68,
            13,    70,  1539,   705,     3,    16,    11, 40271,  1510, 27691,
           198,   220,   220,   220,   220]], device='cuda:0')


The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


\generated_text :   If the document is not correct, it is not correct.
     If the document is not correct, it is not correct.
     If the document is not correct, it is not correct.
     If the document is not correct, it is not correct.
      If the document is not correct, it is not correct.
                 
\input_ids : tensor([[24361,    25,  1867,   373,   262, 25333,   290, 13598,  1459, 25333,
          3292,  1414,  2977, 15278, 25333, 27255,  3176, 12834,  1459,  3739,
          1687, 25333,   555, 39123, 13089,  6538,  4414, 13675,  8617,   584,
          1459, 25333,   287,  1160,  1954,    30,   198,   220,   220,   220,
         29750, 23998,    25,  1114,   262,   614,  1160,  1954,    11,   262,
         25333,   290, 13598,  1459, 25333,  3292,  1414,  2977, 15278, 25333,
         27255,  3176, 12834,  1459,  3739,  1687, 25333,   555, 39123, 13089,
          6538,  4414, 13675,  8617,   584,  1459, 25333,   373,   720,    17,
            11, 31552,  1510,    13,   19

In [24]:
import json
import logging
import os
import torch
import re
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForCausalLM
import time
from torch.optim import AdamW
from tabulate import tabulate

# Attempt to import and mount Google Drive, handle error if not in Colab
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    base_dir = '/content/drive/My Drive/qa'  # Adjust if your folder structure is different
    IN_COLAB = True
except ImportError:
    base_dir = '.' # Default to current directory if not in Colab
    IN_COLAB = False


# -------------------------
# 1. Configuration
# -------------------------
MODEL_NAME = "distilgpt2"
FINETUNED_MODEL_PATH = os.path.join(base_dir, "distilgpt2-finetuned")


# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
logger = logging.getLogger(__name__)

# -------------------------
# 2. Custom Dataset Class
# -------------------------
class QADataset(Dataset):
    """Custom dataset for loading Q&A text file."""
    def __init__(self, tokenizer, file_path, block_size):
        logger.info(f"Loading and tokenizing data from {file_path}")
        with open(file_path, 'r', encoding='utf-8') as f:
            text = f.read()

        tokenized_text = tokenizer.encode(text)

        self.examples = []
        for i in range(0, len(tokenized_text) - block_size + 1, block_size):
            self.examples.append(torch.tensor(tokenized_text[i:i + block_size], dtype=torch.long))
        logger.info(f"Created {len(self.examples)} examples.")

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, i):
        return self.examples[i]

# -------------------------
# 3. Data Preparation
# -------------------------
def prepare_finetuning_data(qa_file_path, output_txt_path):
    """Converts the Q&A JSON file into a single text file for fine-tuning."""
    logger.info(f"Preparing data from {qa_file_path} for fine-tuning.")
    try:
        with open(qa_file_path, 'r', encoding='utf-8') as f:
            qa_pairs = json.load(f)

        formatted_text = ""
        for pair in qa_pairs:
            instruction = "You are a financial analyst bot. Your task is to answer questions based on the provided financial data."
            input_text = pair.get('question') or pair.get('input')
            output_text = pair.get('answer') or pair.get('output')

            if not input_text or not output_text:
                continue

            formatted_text += f"### Instruction:\n{instruction}\n\n### Input:\n{input_text}\n\n### Response:\n{output_text}\n\n"

        with open(output_txt_path, 'w', encoding='utf-8') as f:
            f.write(formatted_text)

        logger.info(f"Successfully created fine-tuning data at {output_txt_path}")
        return True
    except FileNotFoundError:
        logger.error(f"Error: The file was not found at {qa_file_path}")
        return False
    except Exception as e:
        logger.error(f"Failed to prepare fine-tuning data: {e}")
        return False

# -------------------------
# 4. Fine-Tuning Function
# -------------------------
def fine_tune_model(model_name, finetune_txt_path, output_dir):
    """Fine-tunes the model on the prepared dataset."""
    logger.info(f"--- Starting Fine-Tuning for model '{model_name}' ---")

    # --- Hyperparameters ---
    learning_rate = 1e-5
    batch_size = 4
    num_epochs = 10

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    logger.info(f"Compute Setup: Using device: {device}")
    logger.info(f"Hyperparameters: LR={learning_rate}, Batch Size={batch_size}, Epochs={num_epochs}")

    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    dataset = QADataset(tokenizer, file_path=finetune_txt_path, block_size=128)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    optimizer = AdamW(model.parameters(), lr=learning_rate)

    model.train()
    for epoch in range(num_epochs):
        logger.info(f"--- Epoch {epoch + 1}/{num_epochs} ---")
        total_loss = 0
        for i, batch in enumerate(dataloader):
            inputs = batch.to(device)

            outputs = model(inputs, labels=inputs)
            loss = outputs.loss

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            if i % 10 == 0:
                logger.info(f"  Batch {i}/{len(dataloader)}, Loss: {loss.item():.4f}")

        avg_loss = total_loss / len(dataloader)
        logger.info(f"--- End of Epoch {epoch + 1}, Average Loss: {avg_loss:.4f} ---")

    logger.info("Fine-tuning complete. Saving model...")
    model.save_pretrained(output_dir)
    tokenizer.save_pretrained(output_dir)
    logger.info(f"Model saved to {output_dir}")


# -------------------------
# 5. Evaluation Function
# -------------------------
def evaluate_model(model_path, test_questions):
    """Evaluates a fine-tuned model on the test questions."""
    logger.info(f"--- Starting Evaluation for model at '{model_path}' ---")

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    logger.info(f"Using device: {device}")

    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = AutoModelForCausalLM.from_pretrained(model_path).to(device)

    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    logger.info("\n--- Generating answers to test questions ---")
    results_data = []
    total_correct = 0
    for item in test_questions:
        question = item["question"]
        expected_answer = item["answer"]

        prompt = f"""Question: {question}
    Provided Answer: {expected_answer}
    Document Context: {test_questions[:2000]}...

    Extract or generate the correct ground truth value from the document that matches the question.
    If mismatched, correct it based on the document. Output only the value (e.g., '1481').
    """
        inputs = tokenizer(prompt, return_tensors="pt").to(device)
        input_ids = inputs['input_ids']

        start_time = time.time()
        outputs = model.generate(
            input_ids=input_ids,
            max_new_tokens=100,
            temperature=0.7,
            top_k=50,
            top_p=0.95,
            num_return_sequences=1,
            pad_token_id=tokenizer.eos_token_id,
            output_scores=True,
            return_dict_in_generate=True
        )
        end_time = time.time()

        generated_ids = outputs.sequences[:, input_ids.shape[-1]:]
        generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

        scores = torch.stack(outputs.scores, dim=1).log_softmax(-1)
        generated_token_logprobs = scores.gather(2, generated_ids[:, :, None]).squeeze(-1)
        avg_prob = generated_token_logprobs.mean().exp().item() * 100

        is_correct = False
        expected_numbers = re.findall(r'\$([\d,]+)', expected_answer)
        if expected_numbers:
            if any(num in generated_text for num in expected_numbers):
                is_correct = True
                total_correct += 1

        accuracy_score = 1 if is_correct else 0

        results_data.append([
            question,
            expected_answer,
            generated_text,
            f"{avg_prob:.2f}%",
            accuracy_score,
            f"{end_time - start_time:.4f}"
        ])

    headers = ["Question", "Expected Answer", "Generated Answer", "Confidence", "Accuracy (1/0)", "Inference Speed (s)"]
    print(tabulate(results_data, headers=headers, tablefmt="grid"))

    overall_accuracy = (total_correct / len(test_questions)) * 100
    logger.info(f"\nOverall Accuracy: {overall_accuracy:.2f}% ({total_correct}/{len(test_questions)})")


# -------------------------
# 6. Main Execution
# -------------------------
def main():
    """Main function to run the fine-tuning pipeline."""
    qa_json_path = os.path.join(base_dir, "financial_qa_pairs.json")
    finetune_txt_path = os.path.join(base_dir, "finetuning_data.txt")

    logger.info(f"Pipeline started with QA JSON path: {qa_json_path}")

    if not os.path.exists(qa_json_path):
        logger.error(f"The source file '{qa_json_path}' was not found.")
        return


    # Step 2: Fine-tune the model
    fine_tune_model(MODEL_NAME, finetune_txt_path, FINETUNED_MODEL_PATH)

    # Step 3: Evaluate the fine-tuned model
    test_questions = [
        {"question": "What was the assets current assets cash and cash equivalents in 2023?", "answer": "For the year 2023, the assets current assets cash and cash equivalents was $1,481 million."},
        {"question": "What was the current investments trade receivables in 2023?", "answer": "For the year 2023, the current investments trade receivables was $3,094 million."},
        {"question": "What was the unbilled revenues in 2023?", "answer": "For the year 2023, the unbilled revenues was $1,861 million."},
        {"question": "What was the prepayments and other current assets in 2023?", "answer": "For the year 2023, the prepayments and other current assets was $1,336 million."},
        {"question": "What was the income tax assets derivative financial instruments total current assets in 2023?", "answer": "For the year 2023, the income tax assets derivative financial instruments total current assets was $8,626 million."},
        {"question": "What was the non-current assets property, plant and equipment in 2023?", "answer": "For the year 2023, the non-current assets property, plant and equipment was $1,679 million."},
        {"question": "What was the right-of-use assets goodwill intangible assets non-current investments in 2023?", "answer": "For the year 2023, the right-of-use assets goodwill intangible assets non-current investments was $1,530 million."},
        {"question": "What was the unbilled revenues deferred income tax assets income tax assets other non-current assets total non-current assets in 2023?", "answer": "For the year 2023, the unbilled revenues deferred income tax assets income tax assets other non-current assets total non-current assets was $6,686 million."},
        {"question": "What was the total assets in 2023?", "answer": "For the year 2023, the total assets was $15,312 million."},
        {"question": "What was the liabilities and equity current liabilities trade payables lease liabilities derivative financial instruments current income tax liabilities unearned revenues employee benefit obligations provisions other current liabilities in 2023?", "answer": "For the year 2023, the liabilities and equity current liabilities trade payables lease liabilities derivative financial instruments current income tax liabilities unearned revenues employee benefit obligations provisions other current liabilities was $2,403 million."}
    ]

    evaluate_model(FINETUNED_MODEL_PATH, test_questions)


if __name__ == "__main__":
    main()


Mounted at /content/drive


Token indices sequence length is longer than the specified maximum sequence length for this model (3821 > 1024). Running this sequence through the model will result in indexing errors
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANS

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------+--------------+------------------+-----------------------+
| Question                                                                                                                                                                                                                                              | Expected Answer                                                             

In [2]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.7 MB/s eta 0:00:00




### > **Evaluate ditil on Bench mark**



In [40]:
import json

from transformers import pipeline
import json
import logging
import os
import torch
import re
from torch.utils.data import Dataset, DataLoader
import time
from torch.optim import AdamW
from tabulate import tabulate
import time
import torch
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, pipeline


# Attempt to import and mount Google Drive, handle error if not in Colab
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
pre_tuned_data = '/content/drive/My Drive/data/qa/financial_tuned.json'  # Adjust if your folder structure is different
IN_COLAB = True
qa_data_dir = '/content/drive/My Drive/data/qa/financial_qa_pairs.json'


# -------------------------
# 1. Config
# -------------------------
MODEL_NAME = "distilbert-base-uncased"

''' qa_data = [
    {"question": "What was the assets current assets cash and cash equivalents in 2023?", "answer": "For the year 2023, the assets current assets cash and cash equivalents was $1,481 million."},
    {"question": "What was the current investments trade receivables in 2023?", "answer": "For the year 2023, the current investments trade receivables was $3,094 million."},
    {"question": "What was the unbilled revenues in 2023?", "answer": "For the year 2023, the unbilled revenues was $1,861 million."},
    {"question": "What was the prepayments and other current assets in 2023?", "answer": "For the year 2023, the prepayments and other current assets was $1,336 million."},
    {"question": "What was the income tax assets derivative financial instruments total current assets in 2023?", "answer": "For the year 2023, the income tax assets derivative financial instruments total current assets was $8,626 million."},
] '''

# --------------------------
# 2. Load context
# --------------------------
input_files = [
    "/content/drive/My Drive/output/infosys_2023_balance_sheet.txt",
    "/content/drive/My Drive/output/infosys_2024_balance_sheet.txt",
    "/content/drive/My Drive/output/infosys_2023_income_statement.txt",
    "/content/drive/My Drive/output/infosys_2024_income_statement.txt"
]

with open(input_files[0], "r", encoding="utf-8") as f:
    balance_2023 = f.read()
with open(input_files[2], "r", encoding="utf-8") as f:
    income_2023 = f.read()
with open(input_files[1], "r", encoding="utf-8") as f:
    balance_2024 = f.read()
with open(input_files[3], "r", encoding="utf-8") as f:
    income_2024 = f.read()


context = balance_2023 + "\n" + income_2023 + "\n" + balance_2024 + "\n" + income_2024
def prepare_finetuning_data(qa_file_path, output_txt_path):
    """Converts the Q&A JSON file into a single text file for fine-tuning."""
    all_data=[]
    try:
        with open(qa_file_path, 'r', encoding='utf-8') as f:
            qa_pairs = json.load(f)

        # 2. Loop through your qa_pairs
        for pair in qa_pairs:
            # Create a dictionary for each entry
            entry = {
                "question": pair['question'],
                "answer": pair['answer'],
                "context": context
            }
            # Add the dictionary to your list
            all_data.append(entry)

        with open(output_txt_path, 'w', encoding='utf-8') as f:
            json.dump(all_data, f, indent=4)
        return formatted_string
    except Exception as e:
        return None
# Step 1: Prepare the data
formatted_string=prepare_finetuning_data(qa_data_dir, pre_tuned_data)

# --------------------------
# 2. Load tokenizer & model
# --------------------------
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForQuestionAnswering.from_pretrained(MODEL_NAME)

# QA pipeline (will run poorly because model is not fine-tuned)
qa_model = pipeline("question-answering", model=model, tokenizer=tokenizer)

# --------------------------
# 4. Evaluate
# --------------------------
logs = []
for i, qa in enumerate(qa_data):
    if i >= 10:   # stop after 10th iteration
        break
    start_time = time.time()
    result = qa_model(question=qa["question"], context=context)
    elapsed = round(time.time() - start_time, 4)

    logs.append({
        "question": qa["question"],
        "true_answer": qa["answer"],
        "predicted": result["answer"],
        "confidence": result.get("score", None),
        "inference_time_sec": elapsed
    })

# --------------------------
# 5. Print results
# --------------------------
# --------------------------
# 5. Print results in table
# --------------------------
table_data = []
for log in logs:
    table_data.append([
        log['question'],
        log['true_answer'],
        log['predicted'],
        round(log['confidence'], 4) if log['confidence'] else None,
        log['inference_time_sec']
    ])

headers = ["Question", "True Answer", "Predicted Answer", "Confidence", "Inference Time (s)"]

print(tabulate(table_data, headers=headers, tablefmt="grid"))

Mounted at /content/drive


Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cuda:0


+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+--------------+----------------------+
| Question                                                                                                                                                                                                                                              | True Answer                                                                                                                                                                 

In [46]:
import json
import time
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, pipeline, TrainingArguments, Trainer

from tabulate import tabulate
import random
from datasets import Dataset, DatasetDict

# -------------------------
# Config
# -------------------------
MODELS = [
    "distilbert-base-uncased",         # encoder-only base
    "microsoft/MiniLM-L12-H384-uncased" # smaller, faster encoder-only
    # you can add more here
]

qa_data_dir = '/content/drive/My Drive/data/qa/financial_qa_pairs.json'

with open(qa_data_dir, 'r', encoding='utf-8') as f:
    qa_data = json.load(f)

# --------------------------
# Load context (same as before)
# --------------------------
input_files = [
    "/content/drive/My Drive/output/infosys_2023_balance_sheet.txt",
    "/content/drive/My Drive/output/infosys_2024_balance_sheet.txt",
    "/content/drive/My Drive/output/infosys_2023_income_statement.txt",
    "/content/drive/My Drive/output/infosys_2024_income_statement.txt"
]

context = ""
for file in input_files:
    with open(file, "r", encoding="utf-8") as f:
        context += f.read() + "\n"

# --------------------------
# Evaluate each model
# --------------------------
all_results = {}

for model_name in MODELS:
    print(f"\n=== Evaluating Model: {model_name} ===")
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForQuestionAnswering.from_pretrained(model_name)

    qa_model = pipeline("question-answering", model=model, tokenizer=tokenizer)

    logs = []
    for i, qa in enumerate(qa_data):
        if i >= 10:   # only first 10 questions
            break
        start_time = time.time()
        result = qa_model(question=qa["question"], context=context)
        elapsed = round(time.time() - start_time, 4)

        logs.append({
            "question": qa["question"],
            "true_answer": qa["answer"],
            "predicted": result["answer"],
            "confidence": result.get("score", None),
            "inference_time_sec": elapsed
        })

    # Store results for this model
    all_results[model_name] = logs

    # Pretty print results
    table_data = []
    for log in logs:
        table_data.append([
            log['question'],
            log['true_answer'],
            log['predicted'],
            round(log['confidence'], 4) if log['confidence'] else None,
            log['inference_time_sec']
        ])

    headers = ["Question", "True Answer", "Predicted Answer", "Confidence", "Inference Time (s)"]
    print(tabulate(table_data, headers=headers, tablefmt="grid"))



=== Evaluating Model: distilbert-base-uncased ===


Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cuda:0


+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+--------------+----------------------+
| Question                                                                                                                                                                                                                                              | True Answer                                                                                                                                                                 

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at microsoft/MiniLM-L12-H384-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cuda:0


+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------+--------------+----------------------+
| Question                                                                                                                                                                                                                                              | True Answer                                                                                                                                                

In [4]:
import os
import json
import time
import torch
import random
import logging
import re
from datasets import Dataset, DatasetDict
from transformers import (
    AutoTokenizer,
    AutoModelForQuestionAnswering,
    TrainingArguments,
    Trainer,
    default_data_collator,
    pipeline,
)

# -------------------------
# 1) Config & Logging
# -------------------------
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
MODEL_NAME = "distilbert-base-uncased"                # Base model (encoder-only)
qa_data_path_finetuned = "/content/drive/My Drive/data/qa/financial_tuned.json"
model_fine_tuned_dir   = "/content/drive/My Drive/model/qa_distilbert"

LEARNING_RATE = 3e-5
BATCH_SIZE = 8
EPOCHS = 3
MAX_LENGTH = 384
DOC_STRIDE = 128

device = "cuda" if torch.cuda.is_available() else "cpu"
logger.info(f"Using device: {device}")
input_files = [
    "/content/drive/My Drive/data/segmented/infosys_2023_balance_sheet.txt",
    "/content/drive/My Drive/data/segmented/infosys_2024_balance_sheet.txt",
    "/content/drive/My Drive/data/segmented/infosys_2023_income_statement.txt",
    "/content/drive/My Drive/data/segmented/infosys_2024_income_statement.txt"
]

financial_context = ""
for file in input_files:
    with open(file, "r", encoding="utf-8") as f:
        text = f.read()
        # Convert table-like rows to sentence-like statements
        text = re.sub(r"(\d[\d,\.]*\s*[\w$%]*)", lambda m: m.group(0).replace(",", ""), text)
        lines = text.split("\n")
        for line in lines:
            if line.strip():
                financial_context += line.strip() + ". "
# -------------------------
# 2) Load Q/A Dataset (expects dicts with: question, answer, context)
# -------------------------
with open(qa_data_path_finetuned, "r", encoding="utf-8") as f:
    qa_data = json.load(f)

random.shuffle(qa_data)
split_idx = int(0.8 * len(qa_data))
train_data = qa_data[:split_idx]
val_data   = qa_data[split_idx:]

train_dataset = Dataset.from_list(train_data)
val_dataset   = Dataset.from_list(val_data)
dataset = DatasetDict({"train": train_dataset, "validation": val_dataset})

# -------------------------
# 3) Tokenizer
# -------------------------
# DistilBERT does not use token_type_ids; we suppress them in preprocessing.
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)

# -------------------------
# 4) Add SQuAD-style answers (answer text + start char)
# -------------------------
def add_answer_positions(example):
    context = example["context"]
    answer  = example["answer"]

    # Normalize numeric answers
    answer_clean = re.sub(r"[^\d\.-]", "", answer)
    start_idx = context.find(answer)
    if start_idx == -1:
        example["answers"] = {"text": [answer_clean], "answer_start": [0]}
    else:
        example["answers"] = {"text": [answer_clean], "answer_start": [start_idx]}
    return example

dataset = dataset.map(add_answer_positions)

# -------------------------
# 5) Tokenize with offset mapping to align spans
# -------------------------
def preprocess_training_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    contexts  = examples["context"]
    answers   = examples["answers"]

    tokenized = tokenizer(
        questions,
        contexts,
        truncation="only_second",
        max_length=MAX_LENGTH,
        stride=DOC_STRIDE,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
        return_token_type_ids=False,
    )

    start_positions = []
    end_positions   = []

    for i, offsets in enumerate(tokenized["offset_mapping"]):
        sample_idx = tokenized["overflow_to_sample_mapping"][i]
        answer     = answers[sample_idx]
        start_char = answer["answer_start"][0]
        answer_text = answer["text"][0]
        end_char = start_char + len(answer_text)

        sequence_ids = tokenized.sequence_ids(i)

        # Context start/end
        ctx_start = next((idx for idx, s in enumerate(sequence_ids) if s == 1), 0)
        ctx_end   = len(sequence_ids) - 1 - next((idx for idx, s in enumerate(reversed(sequence_ids)) if s == 1), 0)

        if answer_text == "" or start_char == 0:
            cls_index = tokenized["input_ids"][i].index(tokenizer.cls_token_id)
            start_positions.append(cls_index)
            end_positions.append(cls_index)
        else:
            token_start_index = ctx_start
            while token_start_index <= ctx_end and offsets[token_start_index] and offsets[token_start_index][0] <= start_char:
                token_start_index += 1
            start_token = token_start_index - 1

            token_end_index = ctx_end
            while token_end_index >= ctx_start and offsets[token_end_index] and offsets[token_end_index][1] >= end_char:
                token_end_index -= 1
            end_token = token_end_index + 1

            start_positions.append(start_token)
            end_positions.append(end_token)

        tokenized["offset_mapping"][i] = [(o if sequence_ids[k] == 1 else None) for k, o in enumerate(offsets)]

    tokenized["start_positions"] = start_positions
    tokenized["end_positions"]   = end_positions
    return tokenized

tokenized_datasets = dataset.map(
    preprocess_training_examples,
    batched=True,
    remove_columns=dataset["train"].column_names,
)

# -------------------------
# 6) Model
# -------------------------
model = AutoModelForQuestionAnswering.from_pretrained(MODEL_NAME)

# -------------------------
# 7) Training setup
# -------------------------
training_args = TrainingArguments(
    output_dir=model_fine_tuned_dir,
    eval_strategy="epoch",
    learning_rate=LEARNING_RATE,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=EPOCHS,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=50,
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    report_to="none",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=default_data_collator,
    # compute_metrics=compute_metrics,  # Optional
)

# -------------------------
# 8) Train
# -------------------------
trainer.train()

# -------------------------
# 9) Save model & tokenizer
# -------------------------
#os.makedirs(model_fine_tuned_dir, exist_ok=True)
model.save_pretrained(model_fine_tuned_dir)
tokenizer.save_pretrained(model_fine_tuned_dir)

# -------------------------
# 10) Inference (DistilBERT: no token_type_ids)
# -------------------------
qa_pipeline = pipeline(
    "question-answering",
    model=model_fine_tuned_dir,
    tokenizer=tokenizer,
    device=0 if device=="cuda" else -1,
)

def numeric_postprocess(answer):
    # Keep only digits, minus, dot
    clean = re.sub(r"[^\d\.-]", "", answer)
    return clean

# Example questions
qa_examples = [
    {"question": "What was the assets current assets cash and cash equivalents in 2023?", "context": financial_context},
    {"question": "What was the current investments trade receivables in 2023?", "context": financial_context},
]

for i, ex in enumerate(qa_examples, 1):
    pred = qa_pipeline(question=ex["question"], context=ex["context"])
    clean_ans = numeric_postprocess(pred["answer"])
    print(f"Q{i}: {ex['question']}")
    print("Raw Prediction:", pred["answer"])
    print("Cleaned Numeric Answer:", clean_ans, "\n")


Mounted at /content/drive


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-1858597869.py:184: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
1,1.372600,0.002980
2,0.003500,0.001610
3,0.002300,0.001424


Device set to use cuda:0


Q1: What was the assets current assets cash and cash equivalents in 2023?
Raw Prediction: ,
Cleaned Numeric Answer:  

Q2: What was the current investments trade receivables in 2023?
Raw Prediction: ,
Cleaned Numeric Answer:  



In [8]:
import os
import json
import time
import torch
import random
import re
import logging
from datasets import Dataset, DatasetDict
from transformers import (
    AutoTokenizer,
    AutoModelForQuestionAnswering,
    TrainingArguments,
    Trainer,
    default_data_collator,
    pipeline,
)

# -------------------------
# 1) Config & Logging
# -------------------------
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)
# Attempt to import and mount Google Drive, handle error if not in Colab
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
MODEL_NAME = "distilbert-base-uncased"                # Base model (encoder-only)
qa_data_path_finetuned = "/content/drive/My Drive/data/qa/financial_tuned.json"
model_fine_tuned_dir   = "/content/drive/My Drive/model/qa_distilbert"

LEARNING_RATE = 3e-5
BATCH_SIZE = 8
EPOCHS = 3
MAX_LENGTH = 384
DOC_STRIDE = 128

device = "cuda" if torch.cuda.is_available() else "cpu"
logger.info(f"Using device: {device}")
input_files = [
    "/content/drive/My Drive/data/segmented/infosys_2023_balance_sheet.txt",
    "/content/drive/My Drive/data/segmented/infosys_2024_balance_sheet.txt",
    "/content/drive/My Drive/data/segmented/infosys_2023_income_statement.txt",
    "/content/drive/My Drive/data/segmented/infosys_2024_income_statement.txt"
]

financial_context = ""
for file in input_files:
    with open(file, "r", encoding="utf-8") as f:
        text = f.read()
        # Convert table-like rows to sentence-like statements
        text = re.sub(r"(\d[\d,\.]*\s*[\w$%]*)", lambda m: m.group(0).replace(",", ""), text)
        lines = text.split("\n")
        for line in lines:
            if line.strip():
                financial_context += line.strip() + ". "
print(financial_context)

Mounted at /content/drive
Consolidated Balance Sheet as of March 31 (Dollars in millions except equity share data) Note ASSETS Current assets Cash and cash equivalents 1481 2,305 Current investments Trade receivables 3094 2,995 Unbilled revenues 1861 1,526 Prepayments and other current assets 1336 1,133 Income tax assets Derivative financial instruments Total current assets 8626 8,865 Non-current assets Property, plant and equipment 1679 1,793 Right-of-use assets Goodwill Intangible assets Non-current investments 1530 1,801 Unbilled revenues Deferred income tax assets Income tax assets Other non-current assets Total non-current assets 6686 6,690 Total assets 15312 15,555 LIABILITIES AND EQUITY Current liabilities Trade payables Lease liabilities Derivative financial instruments Current income tax liabilities Unearned revenues Employee benefit obligations Provisions Other current liabilities 2403 2,170 Total current liabilities 4769 4,433 Non-current liabilities Lease liabilities Deferr